# toolkit

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

import time
from tqdm import tqdm

# helper-class

## driver-class

In [2]:
class Driver():

    def __init__(self):
        self._driver_path = "E:\Downloads\chromedriver_win32_105\chromedriver.exe"
        self.driver = webdriver.Chrome(executable_path = self._driver_path)
        # self.driver.implicitly_wait(10)

## page-navigator-class

In [3]:
class Home_page():
    def __init__(self, driver):
        self.fetch_url = 'http://nhb.gov.in/OnlineClient/MISDailyReport.aspx'

        self.category_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[2]/td[2]/select'
        self.center_selectAll_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[6]/td[4]/input[1]'
        self.center_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[2]/td[4]/select'
        self.crop_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[3]/td[2]/select'
        self.date_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[4]/td[2]/div/div[2]/input'
        self.search_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td/input[1]'

        self.driver = driver


    def load(self):
        driver = self.driver
        
        driver.get(self.fetch_url)

        return driver


    def fill_category(self, category_name):
        driver = self.driver
        # category_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[2]/td[2]/select'

        # category_selector = Select(driver.find_element(by = By.XPATH, value = self.category_xpath))
        try:
            category_selector = Select(WebDriverWait(driver, 5).until(EC.staleness_of(driver.find_element(By.XPATH, self.category_xpath))))
        except:
            category_selector = Select(driver.find_element(by = By.XPATH, value = self.category_xpath))
        if (category_selector.first_selected_option.text == category_name):
            return driver

        category_selector.select_by_visible_text(category_name)

        try:
            crop_selector_element = WebDriverWait(driver = driver, timeout = 10)\
                .until(
                    EC.staleness_of(driver.find_element(by = By.XPATH, value = self.crop_xpath))
                )
        except NoSuchElementException:
            crop_selector_element = None


        return driver
    

    def fill_centers(self, center_names = None, selectAll = True):
        driver = self.driver
        # center_selectAll_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[6]/td[4]/input[1]'
        # center_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[2]/td[4]/select'


        if isinstance(center_names, type(None)):
            selectAll = True


        center_selector = Select(driver.find_element(by = By.XPATH, value = self.center_xpath))
        # print('\tselector_made-------------------')
        selected_centers = []
        if selectAll == True:
            center_names = [opt.text for opt in center_selector.options]
        
        center_list = center_selector.all_selected_options
        # print('gotselectedoptions---------------//////', len(center_list))
        selected_centers = [opt.text for opt in center_selector.all_selected_options]
        # print("\t\tselected_centers secured-------------------")
        if (sorted(center_names) == sorted(selected_centers)):
            # print('\t\tequal, skipped--------------------------------')
            return driver


        if selectAll == True:
            center_selector = driver.find_element(by = By.XPATH, value = self.center_selectAll_xpath)
            center_selector.click()
            # print('\t\tclicked------------------------')
        else:
            for center_i in center_names:
                center_selector.select_by_visible_text(center_i)
            # print('\t\tfilled---------------------------')
        

        try:
            crop_selector_element = WebDriverWait(driver = driver, timeout = 10)\
                .until(
                    EC.staleness_of(driver.find_element(by = By.XPATH, value = self.crop_xpath))
                )
        except NoSuchElementException:
            crop_selector_element = None
        
        # print('\t\tdone---------------------')

        return driver
    

    def fill_crop(self, crop_name):
        driver = self.driver
        # crop_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[3]/td[2]/select'
        
        
        crop_selector = Select(driver.find_element(by = By.XPATH, value = self.crop_xpath))
        if (crop_selector.first_selected_option.text == crop_name):
            return driver
        
        crop_selector.select_by_visible_text(crop_name)

        return driver


    def fill_date(self, date_str):
        driver = self.driver
        # date_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[1]/tbody/tr[4]/td[2]/div/div[2]/input'
        
        date_filler = driver.find_element(by = By.XPATH, value = self.date_xpath)
        date_filler.clear()
        date_filler.send_keys(date_str)

        date_selector_element = WebDriverWait(driver = driver, timeout = 10)\
            .until(
                EC.visibility_of_element_located((By.XPATH, self.date_xpath))
            )

        return driver
    

    def any_text_present_after_search(self, driver):
        element = driver.find_element(by = By.XPATH, value = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[3]/tbody/tr[3]/td/span[2]')
        if len(element.text) > 0:
            return element
        return False

    def search(self, category_name, crop_name, date_str, center_names = None, selectAll = False):
        self.fill_category(category_name)
        self.fill_centers(center_names, selectAll = selectAll)
        self.fill_crop(crop_name)
        self.fill_date(date_str)

        driver = self.driver
        # search_xpath = '/html/body/form/div[3]/div/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td/input[1]'

        search_selector = WebDriverWait(driver, 5)\
            .until(
                EC.element_to_be_clickable(driver.find_element(by = By.XPATH, value = self.search_xpath))
            )
        search_selector.click()

        crop_selector_element = WebDriverWait(driver = driver, timeout = 10)\
            .until(
                lambda x : self.any_text_present_after_search(driver)
            )
        
        # driver.implicitly_wait(0.5)
        time.sleep(0.75)

        return driver
        

## scraper-class

In [4]:
class scraper():
    def __init__(self):
        self.driver = Driver().driver
        self.homepage = Home_page(self.driver)

        self._init_input_dicts()

        self.ignored_exceptions = (NoSuchElementException,StaleElementReferenceException)
    

    def generate_df(self, category_name, crop_name, date_str, center_names = None, selectAllCenters = False):
        # print("[DEBUG] [generate_df] [started]")
        
        target_page = self.homepage.search(category_name, crop_name, date_str, center_names, selectAllCenters)

        
        response_content = target_page.page_source
        tables = pd.read_html(response_content)
        
        if len(tables) != 8:
            # print("[DEBUG] [generate_df] [ended] : empty form")
            return pd.DataFrame()
        
        raw_df = tables[7]
        
        raw_df['Category'] = category_name
        raw_df['CropName'] = crop_name
        raw_df['SearchDate'] = date_str
        
        raw_df.columns = [i.split('(')[0] for i in raw_df.columns]
        raw_df.drop(columns = 'S.No.', inplace = True)

        # print("[DEBUG] [generate_df] [ended]")
        return raw_df


    def _init_input_dicts(self):
        self.centers_dict = self._generate_centers_dict()
        self.category_dict = self._generate_category_dict()
        self.crop_dict = self._generate_crops_dict()


    def _generate_centers_dict(self):
        print("[DEBUG] [_generate_centers_dict] [started]")
        
        target_page = self.homepage.load()
        home_content = target_page.page_source
        home_soup = BeautifulSoup(home_content, "html.parser")
        # print(home_soup.contents)

        centers_data = home_soup.find("select", id="ctl00_ContentPlaceHolder1_LsboxCenterList")
        centers_options = centers_data.find_all("option")
        
        centers_names = []
        centers_ids = []
        for option_i in centers_options:
            if option_i['value'] == "0":
                continue
            
            center_name = option_i.get_text()
            centers_names.append(center_name)
            
            center_id = option_i["value"]
            centers_ids.append(center_id)

        # print(centers_names)
        centers_dict = dict(zip(centers_names, centers_ids))
        
        print("[DEBUG] [_generate_centers_dict] [ended]")
        return centers_dict
    

    def _generate_category_dict(self):
        print("[DEBUG] [_generate_category_dict] [started]")
        
        target_page = self.homepage.load()
        home_content = target_page.page_source
        home_soup = BeautifulSoup(home_content, "html.parser")
        # print(home_soup.contents)

        category_data = home_soup.find("select", id="ctl00_ContentPlaceHolder1_drpCategoryName")
        category_options = category_data.find_all("option")
        
        category_names = []
        category_ids = []
        for option_i in category_options:
            if option_i['value'] == "0":
                continue
            
            cat_name = option_i.get_text()
            category_names.append(cat_name)
            cat_id = option_i["value"]
            category_ids.append(cat_id)

        # print(category_names)
        category_dict = dict(zip(category_names, category_ids))
        
        print("[DEBUG] [_generate_category_dict] [ended]")
        return category_dict
    

    def _generate_crops_dict(self):
        print("[DEBUG] [_generate_crops_dict] [started]")
        
        category_page = self.homepage.load()
        
        crop_dict = {}
        for category_i in self.category_dict:
            print(f"\t[DEBUG] {category_i} [started]")
            
            target_page = self.homepage.fill_category(category_i)
            crop_content = target_page.page_source
            crop_soup = BeautifulSoup(crop_content, "html.parser")
            # print(crop_soup.contents)

            crop_data = crop_soup.find("select", id="ctl00_ContentPlaceHolder1_drpCropName")
            crop_options = crop_data.find_all("option")
            
            crop_names = []
            crop_ids = []
            for option_i in crop_options:
                if option_i['value'] == "0":
                    continue
                
                crop_name = option_i.get_text()
                crop_names.append(crop_name)
                
                crop_id = option_i["value"]
                crop_ids.append(crop_id)

            # print("\t", crop_names)
            crop_id_dict = dict(zip(crop_names, crop_ids))
            crop_dict[category_i] = crop_id_dict
            
            print(f"\t[DEBUG] {category_i} [ended]")
        
        print("[DEBUG] [_generate_crops_dict] [ended]")
        return crop_dict

## generate-date-list

In [5]:
def get_date_list(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%d/%m/%Y')
    end_date = datetime.strptime(end_date_str, '%d/%m/%Y')

    dates = [i.strftime('%d/%m/%Y') for i in pd.date_range(start=start_date, end=end_date)]
    return dates


# implementation

In [6]:
my_scraper = scraper()

C:\Users\sonis\AppData\Local\Temp\ipykernel_11956\3419089194.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path = self._driver_path)


[DEBUG] [_generate_centers_dict] [started]
[DEBUG] [_generate_centers_dict] [ended]
[DEBUG] [_generate_category_dict] [started]
[DEBUG] [_generate_category_dict] [ended]
[DEBUG] [_generate_crops_dict] [started]
	[DEBUG] FLOWERS [started]
	[DEBUG] FLOWERS [ended]
	[DEBUG] FRUITS [started]
	[DEBUG] FRUITS [ended]
	[DEBUG] VEGETABLES [started]
	[DEBUG] VEGETABLES [ended]
	[DEBUG] Karonda [started]
	[DEBUG] Karonda [ended]
[DEBUG] [_generate_crops_dict] [ended]


In [7]:
my_scraper.crop_dict["VEGETABLES"]

{'BITTER GOURD': '114',
 'BRINJAL': '115',
 'CABBAGE': '116',
 'CAULIFLOWER': '117',
 'GARLIC': '118',
 'GINGER': '119',
 'GREEN CHILLY': '120',
 'OKRA': '121',
 'ONION': '122',
 'PEAS': '123',
 'POTATO': '124',
 'TOMATO': '79'}

In [8]:
custom_category = "FRUITS"
custom_crop = "LITCHI"
custom_dates = get_date_list('01/01/2012', '31/12/2012')

In [9]:
file_name = "./datasets/" + custom_crop + "_" + custom_dates[0].replace('/', '-') + "_" + custom_dates[len(custom_dates) - 1].replace('/', '-') + ".csv"
# file_name

In [10]:
raw_df = pd.DataFrame()
for date_i in tqdm(custom_dates):
    # print(f'date::\t{date_i}')
    try:
        temp_df = my_scraper.generate_df(custom_category, custom_crop, date_i)
    except Exception as e:
        print(e)
        break
    raw_df = pd.concat([raw_df, temp_df], ignore_index = True)


raw_df.to_csv(file_name, index = False)

100%|██████████| 366/366 [43:12<00:00,  7.08s/it]


In [11]:
raw_df

,CentreName,Variety,MinPrice,MaxPrice,ModelPrice,Arrival,RetailPrice,Remark of the Day,Category,CropName,SearchDate
0,HYDERABAD / हैदराबाद,LITCHI,4000,5000,4500,523.0,7500,-,FRUITS,LITCHI,23/01/2012
1,HYDERABAD / हैदराबाद,LITCHI,1700,2500,2200,259.0,2800,-,FRUITS,LITCHI,26/03/2012
2,DELHI / दिल्ली,LITCHI,9000,11000,10000,2.0,12000,-,FRUITS,LITCHI,02/05/2012
3,DELHI / दिल्ली,LITCHI,9000,11000,10000,3.0,12000,-,FRUITS,LITCHI,04/05/2012
4,DELHI / दिल्ली,LITCHI,9000,10500,9600,2.0,11000,-,FRUITS,LITCHI,05/05/2012
...,...,...,...,...,...,...,...,...,...,...,...
503,DELHI / दिल्ली,LITCHI,11000,12500,11500,2.0,13000,-,FRUITS,LITCHI,24/07/2012
504,DELHI / दिल्ली,LITCHI,10500,12000,11000,0.0,13000,-,FRUITS,LITCHI,25/07/2012
505,DELHI / दिल्ली,LITCHI,10500,12000,11000,1.0,13000,-,FRUITS,LITCHI,26/07/2012
506,DELHI / दिल्ली,LITCHI,10500,12000,11000,0.0,12500,-,FRUITS,LITCHI,27/07/2012


In [12]:

raw_df.to_csv(file_name, index = False)

In [13]:
my_scraper.driver.quit()